# Pricing examples using Atlas (No AD)

- Updated: 2023-08-06 
- Status: Active

In [1]:
import Atlas
import Atlas.Visitors as Visitors
import Atlas.Parsers as Parsers
import pandas as pd

In [2]:
evalDate = Atlas.Date(1, Atlas.August, 2020)

#### Initialize market variables

Market variables are stored in the ```MarketStore```, we add the rate curves, rate indexes and fx exchange rates.

In [3]:
evalDate = Atlas.Date(1, Atlas.August, 2020)
store = Atlas.MarketStore(evalDate, Atlas.CLP()) # store with CLP as base currency

# define curve
curveDayCounter = Atlas.Actual360()
curveCompounding = Atlas.Simple
curveFrequency = Atlas.Annual

clpRate = 0.03
usdRate = 0.01
fx = 500


index = Atlas.InterestRateIndex(Atlas.Frequency.Annual)
# add CLP Curve
dates = [evalDate + Atlas.Period(i, Atlas.Years) for i in range(0, 11)]
rates = [1 for i in range(0, 11)]
clpCurve = Atlas.DiscountLogLinearTermStructure(dates, rates)
#clpCurve = Atlas.FlatForwardTermStructure(evalDate, clpRate, curveDayCounter, curveCompounding, curveFrequency)
clpContextIdx = store.curveManager().addCurveContext("CLP", clpCurve, index, Atlas.CLP(), True)

# add USD Curve
usdCurve = Atlas.FlatForwardTermStructure(evalDate, usdRate, curveDayCounter, curveCompounding, curveFrequency)
usdContextIdx = store.curveManager().addCurveContext("USD", usdCurve, index, Atlas.USD(), True)

# add FX
store.fxManager().addExchangeRate(Atlas.CLP(), Atlas.USD(), fx)

In [4]:
clpCurve.discount(evalDate)

1.0

In [5]:
store.curveManager().summary()

idx | name | refDate    | currency     | isRiskFree | hasValidIndex
0   | CLP  | 2020-08-01 | Chilean peso | true       | true         
1   | USD  | 2020-08-01 | U.S. dollar  | true       | true         


#### Create an instrument

We initialize the instrument with the corresponding discount curve id (passing the context parameter). As default, instrument coupon's are set as "local" currency, meaning that their current currency will match the store local currency.

In [6]:
#define interest rate
rateValue = 0.05
dayCounter = Atlas.Thirty360()
compounding = Atlas.Simple
frequency = Atlas.Annual

rate = Atlas.InterestRate(rateValue, dayCounter, compounding, frequency)
# define zero coupon instrument
notional = 100
startDate = evalDate
endDate = Atlas.Date(1, Atlas.August, 2025)
paymentFrequency = Atlas.Semiannual
side = Atlas.Side.Recieve
instrument = Atlas.FixedRateBulletInstrument(startDate, endDate, paymentFrequency, notional, rate, clpContextIdx, side)

AttributeError: type object 'Atlas.Side' has no attribute 'Recieve'

In [ ]:
instrument.cashflows()

#### Inspect the cashflows
We can use the CashflowProfiler visitor to check the instrument cashflows.

In [ ]:
profiler = Visitors.InstrumentCashflowsConstVisitor()
profiler.visit(instrument)
results = profiler.getResults()
pd.DataFrame({'disbursements': results.disbursements, 'redemptions': results.redemptions, 'fixedRateCoupons': results.fixedRateCoupons}).sort_index()

,disbursements,redemptions,fixedRateCoupons
2020-08-01,100.0,NaN,NaN
2021-02-01,NaN,NaN,2.5
2021-08-01,NaN,NaN,2.5
2022-02-01,NaN,NaN,2.5
2022-08-01,NaN,NaN,2.5
2023-02-01,NaN,NaN,2.5
2023-08-01,NaN,NaN,2.5
2024-02-01,NaN,NaN,2.5
2024-08-01,NaN,NaN,2.5
2025-02-01,NaN,NaN,2.5


### Evaluation process

#### 1. Index the instrument
In the indexing phase, market variables are obtained and stored in a MarketRequest object. This information will be used by a Model, which is in charge of producing all the market data needed for later calculations.

In [ ]:
indexer = Visitors.IndexingVisitor()
indexer.visit(instrument)
request = indexer.getResults()

#### 2. Setup a model and simulate market variables
Currently the only model available (SpotMarketDataModel), takes the market information and generates values assuming common linear product's assumptions.

In [ ]:
model = Atlas.SpotMarketDataModel(request, store)
marketData = model.marketData(evalDate)

#### 3. Setup a visitor and evaluate
Visitor are the ones in charge to do evaluations. When visiting, Visitor will execute the precise code needed for each type of instrument.

- Instrument NPV: ```NPVConstVisitor```

This visitor calculates the NPV of each instruments and adds it to an internal variables called npv_, so if it visits many instrument, the value returned by ```results``` will be the sum of each NPV. In the case of a fixed bond, the NPV is being calculated as:

$$NPV^l = \frac{\Sigma_{1}^{N}c_{i}^{f}df^{f}_{i}}{fx^{f/l}}$$

In [ ]:
npvCalculator = Visitors.NPVConstVisitor(marketData)
npvCalculator.visit(instrument)
npv = npvCalculator.getResults()
print("NPV: {:.4f}".format(npv.npv))

NPV: 9.8990


If we want to calculate the insturment duration, we can use the tape (AD). In this case, the duration is being calculated as:

$$Dur = \frac{dNPV}{dr}$$

In [ ]:
sensCalculator = Visitors.SensitivityConstVisitor(marketData)
sensCalculator.visit(instrument)
sens = sensCalculator.getResults()
sensCalculator.reset()
print("Sens to coupon rate: {:.4f}".format(sens.couponSens))

Sens to coupon rate: 4.6212


- Fixed Income Par Rate: ```ParRateConstVisitor```

This visitor calculates the par rate of a given instrument (in this case, rates are not "accumulated" as before). The par rate is calculated, for a fixed rate instruments as follows:

$$r = \argmin_r (\frac{\Sigma_{1}^{T}c_{i}(r) df_{i}}{N} - df_0)^2$$

Where $df_0$ helps bringing the disbursement to the current evaluation date.

In [ ]:
parSolver = Visitors.ParRateConstVisitor(marketData)
parSolver.visit(instrument)
rate = parSolver.getResults()
print("Par Rate: {:.4f}%".format(rate.parRate*100))

Par Rate: 2.8579%


- Fixed Income Z-Spread: ```ZSpreadCalculator```

This visitor calculates the z-spread of a fixed rate instrument (does not apply to other types of instruments). 

$$s = \argmin_s ({\Sigma_{1}^{N}c_{i} df_{i}(s)} - NPV_{target})^2$$

Where $df_i$ will be calculated using the given day counter, compounding and frequency.

In [ ]:
targetNPV = 100
zspreadCalculator = Visitors.ZSpreadConstVisitor(marketData, targetNPV, dayCounter=Atlas.Actual360(), compounding=Atlas.Simple, frequency=Atlas.Annual)
zspreadCalculator.visit(instrument)
zspread = zspreadCalculator.getResults()
print("Z-spread: CLP+{:.2f} bps".format(zspread*10000))

Z-spread: CLP+249.06 bps
